# Installation Guide

This page describes the installation procedure as executed at Arcelor Mittal in Ghent (BE).  The installation is based on the following technology:

- Windows Server (any version above 2003)
- Apache 2.4 for Windows
- Python 3.9 (as used in the AM Python 2022.04 'am2204' distribution used within AM Ghent and other sites)
- Django
- sqlite database (database is just for keeping some local data, not for real data storage)

Note that the installation can be done differently for other sites at Arcelor Mittal.  Basically, prysent was tested already on:

- OS: Linux and Windows
- Databases: sqlite3, Postgres and SQL Server
- Webserver: Apache, Nginx, gunicorn


## 1. Windows + Apache Installation

### 1.1. Starting Remark

Apache, as several other open source projects, is not so Windows minded.  In this case even, there's no official packaging of the httpd server for Windows.  This is done by third party organisations.

### 1.2. Download Apache (httpd server)

On the Apache's download page, you can see that they don't package themselves for Windows.  However, they provide links to some (if not all, don't know) partners where you can download a binary version of the httpd server.

https://httpd.apache.org/docs/current/platform/windows.html#down

I decided to use the "Apache Lounge" *(1)* link and downloaded the software from this page:

https://www.apachelounge.com/download/

The version downloaded was the 64-bit version, version 16, which comes in a zip file.


### 1.3. Installing Apache

The installation is quite simple.  You just unzip the downloaded file and you copy the sub-directory "Apache24" to the c-drive in the root folder.  Hence, Apache can be found under **C:\Apache24**.

To install the Windows service, you can open a command prompt (cmd.exe) and type the following code:

````
cd c:\Apache24\bin
httpd -k install
````

Afterwards, you can start / stop / restart the service via Windows Service Manager

**Note: it's very important that you copy the files to c:\Apache24, the default installation folder on Windows.  If not, you must set a number of things manually to get wsgi working.**

### 1.4. First Run

When Apache starts and Microsoft's Internet Information Services (webserver) is already running, you will get a port conflict on port 80.  Which is the default http port.  In order to test Apache, you can change the default port to 81 via a change in the file: **c:\Apache24\conf\httpd.conf**

````
#
# Listen: Allows you to bind Apache to specific IP addresses and/or
# ports, instead of the default. See also the <VirtualHost>
# directive.
#
# Change this to Listen on specific IP addresses as shown below to
# prevent Apache from glomming onto all bound IP addresses.
#
#Listen 12.34.56.78:80

Listen 81

````

Check if Apache is running by connecting your browser to **http://server:81** if you changed the port, or **http://server** if you did not change the port.  You should see an Apache welcoming screen.

At the same time, you may change the paths of the log files.  C:\Apache24\logs is a good location on Windows.
- ErrorLog
- AccessLog (2 locations)

### 1.6. Checking Apache's version

````
c:\Apache24\bin>httpd -V
Server version: Apache/2.4.54 (Win64)
Apache Lounge VS16 Server built:   Jun 22 2022 09:58:15
Server's Module Magic Number: 20120211:124
Server loaded:  APR 1.7.0, APR-UTIL 1.6.1, PCRE 10.40 2022-04-14
Compiled using: APR 1.7.0, APR-UTIL 1.6.1, PCRE 10.40 2022-04-14
Architecture:   64-bit
Server MPM:     WinNT
  threaded:     yes (fixed thread count)
    forked:     no
Server compiled with....
 -D APR_HAS_SENDFILE
 -D APR_HAS_MMAP
 -D APR_HAVE_IPV6 (IPv4-mapped addresses disabled)
 -D APR_HAS_OTHER_CHILD
 -D AP_HAVE_RELIABLE_PIPED_LOGS
 -D DYNAMIC_MODULE_LIMIT=256
 -D HTTPD_ROOT="/apache"
 -D SUEXEC_BIN="/apache/bin/suexec"
 -D DEFAULT_PIDLOG="logs/httpd.pid"
 -D DEFAULT_SCOREBOARD="logs/apache_runtime_status"
 -D DEFAULT_ERRORLOG="logs/error.log"
 -D AP_TYPES_CONFIG_FILE="conf/mime.types"
 -D SERVER_CONFIG_FILE="conf/httpd.conf"
 ````

Important to check is that APR is available.  This is needed when you compile wsgi in a later stadium.

### 1.7. Checking Apache's modules

````
c:\Apache24\bin>httpd -M
Loaded Modules:
 core_module (static)
 win32_module (static)
 mpm_winnt_module (static)
 http_module (static)
 so_module (static)
 actions_module (shared)
 alias_module (shared)
 allowmethods_module (shared)
 asis_module (shared)
 auth_basic_module (shared)
 authn_core_module (shared)
 authn_file_module (shared)
 authz_core_module (shared)
 authz_groupfile_module (shared)
 authz_host_module (shared)
 authz_user_module (shared)
 autoindex_module (shared)
 cgi_module (shared)
 dir_module (shared)
 env_module (shared)
 include_module (shared)
 isapi_module (shared)
 log_config_module (shared)
 mime_module (shared)
 negotiation_module (shared)
 setenvif_module (shared)
````

As you can see, the mod_wsgi is not defined or loaded.  We have to do this manually.  This is not simple and we need some pre-requisites.

### 1.8. pip install mod_wsgi

mod_wsgi can be installed using pip.  However, it could be that you must first install the Microsoft Build Tools.  In that case, you will get an error including the link where you can download the package.  See the next item to install it.  Afterwards, continue with this point.

````
pip install mod_wsgi
````

Use this link as backup:

https://learn.microsoft.com/en-us/cpp/windows/latest-supported-vc-redist?view=msvc-170


### 1.9. Installing Microsoft Build Tools

Reference: https://montesariel.com/blog/post-3

On some machines, the Microsoft Build Tools will be missing, certainly on Windows Servers.  You must download this package and run the setup.

![](install1.png)


### 1.10. Copy the settings

The settings you need for Apache's httpd.conf file can be retrieved by the following command:

````
c:\Apache24\bin>mod_wsgi-express module-config
````

These are the settings.  You may simply copy them at the end of the httpd.conf file.

````
LoadFile "C:/Users/sidviny/.conda/envs/prysent/python310.dll"
LoadModule wsgi_module "C:/Users/sidviny/.conda/envs/prysent/lib/site-packages/mod_wsgi/server/mod_wsgi.cp310-win_amd64.pyd"
WSGIPythonHome "C:/Users/sidviny/.conda/envs/prysent"
````

### 1.11. Adding libraries to the PYTHONPATH


````
[mpm_winnt:notice] [pid 27532:tid 452] AH00354: Child: Starting 64 worker threads.
[Mon Oct 17 15:30:35.690402 2022] [mpm_winnt:notice] [pid 26676:tid 464] AH00422: Parent: Received shutdown signal -- Shutting down the server.
Apache server interrupted...
Apache server interrupted...
[Mon Oct 17 15:30:37.703563 2022] [mpm_winnt:notice] [pid 27532:tid 452] AH00364: Child: All worker threads have exited.
[Mon Oct 17 15:30:37.783531 2022] [mpm_winnt:notice] [pid 26676:tid 464] AH00430: Parent: Child process 27532 exited successfully.
[Mon Oct 17 15:37:50.382573 2022] [mpm_winnt:notice] [pid 30228:tid 440] AH00455: Apache/2.4.54 (Win64) mod_wsgi/4.9.4 Python/3.10 configured -- resuming normal operations
[Mon Oct 17 15:37:50.383573 2022] [mpm_winnt:notice] [pid 30228:tid 440] AH00456: Apache Lounge VS16 Server built: Jun 22 2022 09:58:15
[Mon Oct 17 15:37:50.383573 2022] [core:notice] [pid 30228:tid 440] AH00094: Command line: 'httpd -d C:/Apache24'
[Mon Oct 17 15:37:50.394574 2022] [mpm_winnt:notice] [pid 30228:tid 440] AH00418: Parent: Created child process 33020
Python path configuration:
  PYTHONHOME = (not set)
  PYTHONPATH = 'C:\Users\sidviny\Development\py-dex;'
  program name = 'python'
  isolated = 0
  environment = 1
  user site = 1
  import site = 1
  sys._base_executable = 'C:\\Apache24\\bin\\httpd.exe'
  sys.base_prefix = 'C:\\Users\\sidviny\\.conda\\envs\\prysent'
  sys.base_exec_prefix = 'C:\\Users\\sidviny\\.conda\\envs\\prysent'
  sys.platlibdir = 'lib'
  sys.executable = 'C:\\Apache24\\bin\\httpd.exe'
  sys.prefix = 'C:\\Users\\sidviny\\.conda\\envs\\prysent'
  sys.exec_prefix = 'C:\\Users\\sidviny\\.conda\\envs\\prysent'
  sys.path = [
    'C:\\Users\\sidviny\\Development\\py-dex',
    '',
    'C:\\Users\\sidviny\\.conda\\envs\\prysent\\python310.zip',
    'C:\\Apache24\\bin',
  ]
Fatal Python error: init_fs_encoding: failed to get the Python codec of the filesystem encoding
Python runtime state: core initialized
ModuleNotFoundError: No module named 'encodings'

Current thread 0x00006a1c (most recent call first):
  <no Python frame>
````

https://stackoverflow.com/questions/24495348/mod-wsgi-importerror-no-module-named-encodings


````
(prysent) c:\Apache24\bin>set PYTHONPATH=%PYTHONPATH%;C:\Users\sidviny\.conda\envs\prysent\Lib;C:\Users\sidviny\.conda\envs\prysent\DLLs

(prysent) c:\Apache24\bin>echo %PYTHONPATH%
C:\Users\sidviny\Development\py-dex;;C:\Users\sidviny\.conda\envs\prysent\Lib;C:\Users\sidviny\.conda\envs\prysent\DLLs

(prysent) c:\Apache24\bin>httpd
````

Maybe it's best to write a little startup script when testing the installation where you set this PYTHONPATH.

### 1.12. Adding the application's path to PYTHONPATH

Similar to the error above, when httpd is started and you call the homepage, you will get an error that Apache can't find the modules of your own application.  You must also add them to your PYTHONPATH.

Most likely you also need to add the "AM Python - Documents" directory to your PYTHONPATH.  This depends on what type of application you are using.

After repeating the startup sequence a couple of times, I ended up adding the following paths to the PYTHONPATH:

- C:\ProgramData\Miniconda3\envs\CONDA_ENV\DLLs
- C:\ProgramData\Miniconda3\envs\CONDA_ENV\Lib
- C:\ProgramData\Miniconda3\envs\CONDA_ENV\libs
- C:\ProgramData\Miniconda3\envs\CONDA_ENV\Lib\site-packages
- C:\PythonApps\APP\src
- C:\Users\XYZ\ArcelorMittal\AM Python - Documenten\PyDex\source\_stable

Below is the example when testing the setup on my personal machine:

````
set PYTHONPATH=C:\Users\sidviny\.conda\envs\prysent\DLLs;C:\Users\sidviny\.conda\envs\prysent\Lib;C:\Users\sidviny\.conda\envs\prysent\libs;C:\Users\sidviny\.conda\envs\prysent\Lib\site-packages;C:\Users\sidviny\Development\prysent\src;C:\Users\sidviny\ArcelorMittal\AM Python - Documenten\PyDex\source\_stable;
````


### 1.13. The last step

After configuring all this, Apache starts up without any problem and you can call the homepage.  Unfortunately, the browser seems to stay in a loop and waits forever.  This is poorly documented and fortunately, I found that one more setting is needed.

Found at: https://github.com/GrahamDumpleton/mod_wsgi/issues/559 (comment on April, 15th 2020)

````
WSGIApplicationGroup %{GLOBAL}
````

At the same time, finding other settings, I added the following lines also:

````
WSGIVerboseDebugging On
WSGIScriptReloading On
````

This finally started my application, although not all is running as it should as my settings file states this is production and hence the static files are not served.  But this is typical to a Django application and the steps to solve this are listed later on.

## 2. Apache Configuration per Application Type

### 2.1.  Apache Virtual Hosts

It's a good idea to create an extra directory "arcelormittal" with in this directory one or multiple files to configure the virtual hosts.  You must do this is Apache has to serve multiple application, but it's a clean way to do it also for one application.

Update the httpd.conf file in case of one file like this:

````
Include conf/arcelormittal/httpd-vhosts.conf
````

Update the httpd.conf file in case of multiple files like this:

````
Include conf/arcelormittal/*.conf
````

Create the extra files and make an entry for your application.  Below are samples for the following types:

- Django
- Flask
- Dash

## 2.2. Django Virtual Host

````
<VirtualHost *:${SRVPORT}>
    #ServerName ws40968.sidmar.be
    #ServerAlias ws40968
    #ServerAlias localhost

    DocumentRoot "${PRYSENTROOT}"

    WSGIScriptAlias / "${PRYSENTROOT}/prysent/wsgi.py"


    ErrorLog "${SRVROOT}/logs/prysent.log"
    CustomLog "${SRVROOT}/logs/prysent.access.log" combined

    Alias /static "${PRYSENTROOT}/_html_cache"

    <Directory "${PRYSENTROOT}/prysent">
        <Files wsgi.py>
            Require all granted
        </Files>
    </Directory>

    <Directory "${PRYSENTROOT}">
        Require all granted
    </Directory>

    <Directory "${PRYSENTROOT}/_html_cache">
        Require all granted
    </Directory>
</VirtualHost>
````

**Note: see the entry for the static files.  This is needed to serve the static files.**

## 2.3. Flask Virtual Host

````
<VirtualHost *:${SRVPORT}>
    #ServerName ws40968.sidmar.be
    #ServerAlias ws40968
    #ServerAlias localhost

    DocumentRoot "${FLASK_APP}"

    WSGIScriptAlias / "${FLASK_APP}/wsgi.py"


    ErrorLog "${SRVROOT}/logs/flask.log"
    CustomLog "${SRVROOT}/logs/flask.access.log" combined


    <Directory "${FLASK_APP}">
        <Files wsgi.py>
            Require all granted
        </Files>
    </Directory>
</VirtualHost>
````

Flask application (hello_world.py):

````
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"
````

WSGI file:

````
from hello_world import app

application = app
````


## 2.4. Dash Virtual Host

````
<VirtualHost *:${SRVPORT}>
    ServerName ws40968.sidmar.be
    ServerAlias ws40968
    ServerAlias localhost

    DocumentRoot "${DASH_APP}"

    WSGIScriptAlias / "${DASH_APP}/wsgi.py"


    ErrorLog "${SRVROOT}/logs/dashboard.log"
    CustomLog "${SRVROOT}/logs/dashboard.access.log" combined


    <Directory "${DASH_APP}">
        <Files wsgi.py>
            Require all granted
        </Files>
    </Directory>
</VirtualHost>
````

index.py (sample from coo_application):

````
"""Script for starting the Dash application.

2018-07-16 - Marzena.Prokopiuk@arcelormittal.com
2021-07-30 - luc.vandeputte@arcelormittal.com
2021-09-29 - ronny.demeyer@arcelormittal.com
"""
# %% Imports
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from app import app
from apps import homepage, benzol, geluid_vw3, geluid_vw4, tijd_deur_open, alarmen

# %% Create the Flask app
server = app.server


# %% Define the layout function
def serve_layout():
    """Show lay-out."""
    return html.Div([dcc.Location(id='url', refresh=False),
                     html.Div(id='page-content'),
                     ])


app.layout = serve_layout

# %% Callbacks (actions to do when an input is changed)
# Apply className to page content based on current pathname


@app.callback(Output('page-content', 'className'),
              [Input('url', 'pathname')])
def pageClassName(pathname):
    """Return page name."""
    className = pathname[1:] if pathname else ''
    return className if className else 'homepage'  # homepage if empty className


@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    """Show page."""
    if pathname == '/':
        return homepage.layout
    elif pathname == '/benzol':
        return benzol.layout
    elif pathname == '/geluid_vw3':
        return geluid_vw3.layout
    elif pathname == '/geluid_vw4':
        return geluid_vw4.layout
    elif pathname == '/tijd_deur_open':
        return tijd_deur_open.layout
    elif pathname == '/alarmen':
        return alarmen.layout
    else:
        return '404 URL not found'


# %% Start the app
if __name__ == '__main__':
    app.run_server(debug=False)
````

wsgi.py:

````
from index import server

application = server
````

## Appendix A: final Apache httpd.conf after local deployment testing

````
#
# This is the main Apache HTTP server configuration file.  It contains the
# configuration directives that give the server its instructions.
# See <URL:http://httpd.apache.org/docs/2.4/> for detailed information.
# In particular, see
# <URL:http://httpd.apache.org/docs/2.4/mod/directives.html>
# for a discussion of each configuration directive.
#
# Do NOT simply read the instructions in here without understanding
# what they do.  They're here only as hints or reminders.  If you are unsure
# consult the online docs. You have been warned.
#
# Configuration and logfile names: If the filenames you specify for many
# of the server's control files begin with "/" (or "drive:/" for Win32), the
# server will use that explicit path.  If the filenames do *not* begin
# with "/", the value of ServerRoot is prepended -- so "logs/access_log"
# with ServerRoot set to "/usr/local/apache2" will be interpreted by the
# server as "/usr/local/apache2/logs/access_log", whereas "/logs/access_log"
# will be interpreted as '/logs/access_log'.
#
# NOTE: Where filenames are specified, you must use forward slashes
# instead of backslashes (e.g., "c:/apache" instead of "c:\apache").
# If a drive letter is omitted, the drive on which httpd.exe is located
# will be used by default.  It is recommended that you always supply
# an explicit drive letter in absolute paths to avoid confusion.

#
# ServerRoot: The top of the directory tree under which the server's
# configuration, error, and log files are kept.
#
# Do not add a slash at the end of the directory path.  If you point
# ServerRoot at a non-local disk, be sure to specify a local disk on the
# Mutex directive, if file-based mutexes are used.  If you wish to share the
# same ServerRoot for multiple httpd daemons, you will need to change at
# least PidFile.
#

# SIDVINY
# Also defining some other consts here

Define SRVROOT "C:/Apache24"
Define SRVPORT 81
Define SRVADMIN "yves.vindevogel.external@arcelormittal.com"
Define SRVLOGLEVEL "info"
Define PRYSENTROOT "C:/Users/sidviny/Development/prysent/src"
Define CONDAROOT "C:/Users/sidviny/.conda/envs/prysent"

ServerRoot "${SRVROOT}"

#
# Mutex: Allows you to set the mutex mechanism and mutex file directory
# for individual mutexes, or change the global defaults
#
# Uncomment and change the directory if mutexes are file-based and the default
# mutex file directory is not on a local disk or is not appropriate for some
# other reason.
#
# Mutex default:logs

#
# Listen: Allows you to bind Apache to specific IP addresses and/or
# ports, instead of the default. See also the <VirtualHost>
# directive.
#
# Change this to Listen on specific IP addresses as shown below to
# prevent Apache from glomming onto all bound IP addresses.
#
#Listen 12.34.56.78:80
Listen ${SRVPORT}

#
# Dynamic Shared Object (DSO) Support
#
# To be able to use the functionality of a module which was built as a DSO you
# have to place corresponding `LoadModule' lines at this location so the
# directives contained in it are actually available _before_ they are used.
# Statically compiled modules (those listed by `httpd -l') do not need
# to be loaded here.
#
# Example:
# LoadModule foo_module modules/mod_foo.so
#
#LoadModule access_compat_module modules/mod_access_compat.so
LoadModule actions_module modules/mod_actions.so
LoadModule alias_module modules/mod_alias.so
LoadModule allowmethods_module modules/mod_allowmethods.so
LoadModule asis_module modules/mod_asis.so
LoadModule auth_basic_module modules/mod_auth_basic.so
#LoadModule auth_digest_module modules/mod_auth_digest.so
#LoadModule auth_form_module modules/mod_auth_form.so
#LoadModule authn_anon_module modules/mod_authn_anon.so
LoadModule authn_core_module modules/mod_authn_core.so
#LoadModule authn_dbd_module modules/mod_authn_dbd.so
#LoadModule authn_dbm_module modules/mod_authn_dbm.so
LoadModule authn_file_module modules/mod_authn_file.so
#LoadModule authn_socache_module modules/mod_authn_socache.so
#LoadModule authnz_fcgi_module modules/mod_authnz_fcgi.so
#LoadModule authnz_ldap_module modules/mod_authnz_ldap.so
LoadModule authz_core_module modules/mod_authz_core.so
#LoadModule authz_dbd_module modules/mod_authz_dbd.so
#LoadModule authz_dbm_module modules/mod_authz_dbm.so
LoadModule authz_groupfile_module modules/mod_authz_groupfile.so
LoadModule authz_host_module modules/mod_authz_host.so
#LoadModule authz_owner_module modules/mod_authz_owner.so
LoadModule authz_user_module modules/mod_authz_user.so
LoadModule autoindex_module modules/mod_autoindex.so
#LoadModule brotli_module modules/mod_brotli.so
#LoadModule buffer_module modules/mod_buffer.so
#LoadModule cache_module modules/mod_cache.so
#LoadModule cache_disk_module modules/mod_cache_disk.so
#LoadModule cache_socache_module modules/mod_cache_socache.so
#LoadModule cern_meta_module modules/mod_cern_meta.so
#LoadModule cgi_module modules/mod_cgi.so
#LoadModule charset_lite_module modules/mod_charset_lite.so
#LoadModule data_module modules/mod_data.so
#LoadModule dav_module modules/mod_dav.so
#LoadModule dav_fs_module modules/mod_dav_fs.so
#LoadModule dav_lock_module modules/mod_dav_lock.so
#LoadModule dbd_module modules/mod_dbd.so
#LoadModule deflate_module modules/mod_deflate.so
#LoadModule dir_module modules/mod_dir.so
#LoadModule dumpio_module modules/mod_dumpio.so
LoadModule env_module modules/mod_env.so
#LoadModule expires_module modules/mod_expires.so
#LoadModule ext_filter_module modules/mod_ext_filter.so
#LoadModule file_cache_module modules/mod_file_cache.so
#LoadModule filter_module modules/mod_filter.so
#LoadModule http2_module modules/mod_http2.so
#LoadModule headers_module modules/mod_headers.so
#LoadModule heartbeat_module modules/mod_heartbeat.so
#LoadModule heartmonitor_module modules/mod_heartmonitor.so
#LoadModule ident_module modules/mod_ident.so
#LoadModule imagemap_module modules/mod_imagemap.so
LoadModule include_module modules/mod_include.so
#LoadModule info_module modules/mod_info.so
LoadModule isapi_module modules/mod_isapi.so
#LoadModule lbmethod_bybusyness_module modules/mod_lbmethod_bybusyness.so
#LoadModule lbmethod_byrequests_module modules/mod_lbmethod_byrequests.so
#LoadModule lbmethod_bytraffic_module modules/mod_lbmethod_bytraffic.so
#LoadModule lbmethod_heartbeat_module modules/mod_lbmethod_heartbeat.so
#LoadModule ldap_module modules/mod_ldap.so
#LoadModule logio_module modules/mod_logio.so
LoadModule log_config_module modules/mod_log_config.so
#LoadModule log_debug_module modules/mod_log_debug.so
#LoadModule log_forensic_module modules/mod_log_forensic.so
#LoadModule lua_module modules/mod_lua.so
#LoadModule macro_module modules/mod_macro.so
#LoadModule md_module modules/mod_md.so
LoadModule mime_module modules/mod_mime.so
#LoadModule mime_magic_module modules/mod_mime_magic.so
LoadModule negotiation_module modules/mod_negotiation.so
#LoadModule proxy_module modules/mod_proxy.so
#LoadModule proxy_ajp_module modules/mod_proxy_ajp.so
#LoadModule proxy_balancer_module modules/mod_proxy_balancer.so
#LoadModule proxy_connect_module modules/mod_proxy_connect.so
#LoadModule proxy_express_module modules/mod_proxy_express.so
#LoadModule proxy_fcgi_module modules/mod_proxy_fcgi.so
#LoadModule proxy_ftp_module modules/mod_proxy_ftp.so
#LoadModule proxy_hcheck_module modules/mod_proxy_hcheck.so
#LoadModule proxy_html_module modules/mod_proxy_html.so
#LoadModule proxy_http_module modules/mod_proxy_http.so
#LoadModule proxy_http2_module modules/mod_proxy_http2.so
#LoadModule proxy_scgi_module modules/mod_proxy_scgi.so
#LoadModule proxy_uwsgi_module modules/mod_proxy_uwsgi.so
#LoadModule proxy_wstunnel_module modules/mod_proxy_wstunnel.so
#LoadModule ratelimit_module modules/mod_ratelimit.so
#LoadModule reflector_module modules/mod_reflector.so
#LoadModule remoteip_module modules/mod_remoteip.so
#LoadModule request_module modules/mod_request.so
#LoadModule reqtimeout_module modules/mod_reqtimeout.so
#LoadModule rewrite_module modules/mod_rewrite.so
#LoadModule sed_module modules/mod_sed.so
#LoadModule session_module modules/mod_session.so
#LoadModule session_cookie_module modules/mod_session_cookie.so
#LoadModule session_crypto_module modules/mod_session_crypto.so
#LoadModule session_dbd_module modules/mod_session_dbd.so
LoadModule setenvif_module modules/mod_setenvif.so
#LoadModule slotmem_plain_module modules/mod_slotmem_plain.so
#LoadModule slotmem_shm_module modules/mod_slotmem_shm.so
#LoadModule socache_dbm_module modules/mod_socache_dbm.so
#LoadModule socache_memcache_module modules/mod_socache_memcache.so
#LoadModule socache_redis_module modules/mod_socache_redis.so
#LoadModule socache_shmcb_module modules/mod_socache_shmcb.so
#LoadModule speling_module modules/mod_speling.so
#LoadModule ssl_module modules/mod_ssl.so
#LoadModule status_module modules/mod_status.so
#LoadModule substitute_module modules/mod_substitute.so
#LoadModule unique_id_module modules/mod_unique_id.so
#LoadModule userdir_module modules/mod_userdir.so
#LoadModule usertrack_module modules/mod_usertrack.so
#LoadModule version_module modules/mod_version.so

# SIDVINY
LoadModule vhost_alias_module modules/mod_vhost_alias.so

#LoadModule watchdog_module modules/mod_watchdog.so
#LoadModule xml2enc_module modules/mod_xml2enc.so

<IfModule unixd_module>
#
# If you wish httpd to run as a different user or group, you must run
# httpd as root initially and it will switch.
#
# User/Group: The name (or #number) of the user/group to run httpd as.
# It is usually good practice to create a dedicated user and group for
# running httpd, as with most system services.
#
User daemon
Group daemon

</IfModule>

# 'Main' server configuration
#
# The directives in this section set up the values used by the 'main'
# server, which responds to any requests that aren't handled by a
# <VirtualHost> definition.  These values also provide defaults for
# any <VirtualHost> containers you may define later in the file.
#
# All of these directives may appear inside <VirtualHost> containers,
# in which case these default settings will be overridden for the
# virtual host being defined.
#

#
# ServerAdmin: Your address, where problems with the server should be
# e-mailed.  This address appears on some server-generated pages, such
# as error documents.  e.g. admin@your-domain.com
#
ServerAdmin "${SRVADMIN}"

#
# ServerName gives the name and port that the server uses to identify itself.
# This can often be determined automatically, but we recommend you specify
# it explicitly to prevent problems during startup.
#
# If your host doesn't have a registered DNS name, enter its IP address here.
#
#ServerName www.example.com:80

#
# Deny access to the entirety of your server's filesystem. You must
# explicitly permit access to web content directories in other
# <Directory> blocks below.
#

# SIDVINY
#<Directory />
#    AllowOverride none
#    Require all denied
#</Directory>

#
# Note that from this point forward you must specifically allow
# particular features to be enabled - so if something's not working as
# you might expect, make sure that you have specifically enabled it
# below.
#

#
# DocumentRoot: The directory out of which you will serve your
# documents. By default, all requests are taken from this directory, but
# symbolic links and aliases may be used to point to other locations.
#
# DocumentRoot "${SRVROOT}/htdocs"
#<Directory "${SRVROOT}/htdocs">
    #
    # Possible values for the Options directive are "None", "All",
    # or any combination of:
    #   Indexes Includes FollowSymLinks SymLinksifOwnerMatch ExecCGI MultiViews
    #
    # Note that "MultiViews" must be named *explicitly* --- "Options All"
    # doesn't give it to you.
    #
    # The Options directive is both complicated and important.  Please see
    # http://httpd.apache.org/docs/2.4/mod/core.html#options
    # for more information.
    #
    #Options Indexes FollowSymLinks

    #
    # AllowOverride controls what directives may be placed in .htaccess files.
    # It can be "All", "None", or any combination of the keywords:
    #   AllowOverride FileInfo AuthConfig Limit
    #
    #AllowOverride None

    #
    # Controls who can get stuff from this server.
    #
    #Require all granted
#</Directory>

#
# DirectoryIndex: sets the file that Apache will serve if a directory
# is requested.
#
<IfModule dir_module>
    DirectoryIndex index.html
</IfModule>

#
# The following lines prevent .htaccess and .htpasswd files from being
# viewed by Web clients.
#
<Files ".ht*">
    Require all denied
</Files>

#
# ErrorLog: The location of the error log file.
# If you do not specify an ErrorLog directive within a <VirtualHost>
# container, error messages relating to that virtual host will be
# logged here.  If you *do* define an error logfile for a <VirtualHost>
# container, that host's errors will be logged there and not here.
#

# SIDVINY
ErrorLog "${SRVROOT}/logs/error.log"

#
# LogLevel: Control the number of messages logged to the error_log.
# Possible values include: debug, info, notice, warn, error, crit,
# alert, emerg.
#
LogLevel "${SRVLOGLEVEL}"

<IfModule log_config_module>
    #
    # The following directives define some format nicknames for use with
    # a CustomLog directive (see below).
    #
    LogFormat "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"" combined
    LogFormat "%h %l %u %t \"%r\" %>s %b" common

    <IfModule logio_module>
      # You need to enable mod_logio.c to use %I and %O
      LogFormat "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\" %I %O" combinedio
    </IfModule>

    #
    # The location and format of the access logfile (Common Logfile Format).
    # If you do not define any access logfiles within a <VirtualHost>
    # container, they will be logged here.  Contrariwise, if you *do*
    # define per-<VirtualHost> access logfiles, transactions will be
    # logged therein and *not* in this file.
    #

	# SIDVINY
    CustomLog "${SRVROOT}/logs/access.log" common

    #
    # If you prefer a logfile with access, agent, and referer information
    # (Combined Logfile Format) you can use the following directive.
    #

	# SIDVINY
    #CustomLog "${SRVROOT}/logs/access.log" combined
</IfModule>

<IfModule alias_module>
    #
    # Redirect: Allows you to tell clients about documents that used to
    # exist in your server's namespace, but do not anymore. The client
    # will make a new request for the document at its new location.
    # Example:
    # Redirect permanent /foo http://www.example.com/bar

    #
    # Alias: Maps web paths into filesystem paths and is used to
    # access content that does not live under the DocumentRoot.
    # Example:
    # Alias /webpath /full/filesystem/path
    #
    # If you include a trailing / on /webpath then the server will
    # require it to be present in the URL.  You will also likely
    # need to provide a <Directory> section to allow access to
    # the filesystem path.

    #
    # ScriptAlias: This controls which directories contain server scripts.
    # ScriptAliases are essentially the same as Aliases, except that
    # documents in the target directory are treated as applications and
    # run by the server when requested rather than as documents sent to the
    # client.  The same rules about trailing "/" apply to ScriptAlias
    # directives as to Alias.
    #
    ScriptAlias /cgi-bin/ "${SRVROOT}/cgi-bin/"
</IfModule>

<IfModule cgid_module>
    #
    # ScriptSock: On threaded servers, designate the path to the UNIX
    # socket used to communicate with the CGI daemon of mod_cgid.
    #
    #Scriptsock cgisock
</IfModule>

#
# "${SRVROOT}/cgi-bin" should be changed to whatever your ScriptAliased
# CGI directory exists, if you have that configured.
#
<Directory "${SRVROOT}/cgi-bin">
    AllowOverride None
    Options None
    Require all granted
</Directory>

<IfModule headers_module>
    #
    # Avoid passing HTTP_PROXY environment to CGI's on this or any proxied
    # backend servers which have lingering "httpoxy" defects.
    # 'Proxy' request header is undefined by the IETF, not listed by IANA
    #
    RequestHeader unset Proxy early
</IfModule>

<IfModule mime_module>
    #
    # TypesConfig points to the file containing the list of mappings from
    # filename extension to MIME-type.
    #
    TypesConfig conf/mime.types

    #
    # AddType allows you to add to or override the MIME configuration
    # file specified in TypesConfig for specific file types.
    #
    #AddType application/x-gzip .tgz
    #
    # AddEncoding allows you to have certain browsers uncompress
    # information on the fly. Note: Not all browsers support this.
    #
    #AddEncoding x-compress .Z
    #AddEncoding x-gzip .gz .tgz
    #
    # If the AddEncoding directives above are commented-out, then you
    # probably should define those extensions to indicate media types:
    #
    AddType application/x-compress .Z
    AddType application/x-gzip .gz .tgz

    #
    # AddHandler allows you to map certain file extensions to "handlers":
    # actions unrelated to filetype. These can be either built into the server
    # or added with the Action directive (see below)
    #
    # To use CGI scripts outside of ScriptAliased directories:
    # (You will also need to add "ExecCGI" to the "Options" directive.)
    #
    #AddHandler cgi-script .cgi

    # For type maps (negotiated resources):
    #AddHandler type-map var

    #
    # Filters allow you to process content before it is sent to the client.
    #
    # To parse .shtml files for server-side includes (SSI):
    # (You will also need to add "Includes" to the "Options" directive.)
    #
    #AddType text/html .shtml
    #AddOutputFilter INCLUDES .shtml
</IfModule>

#
# The mod_mime_magic module allows the server to use various hints from the
# contents of the file itself to determine its type.  The MIMEMagicFile
# directive tells the module where the hint definitions are located.
#
#MIMEMagicFile conf/magic

#
# Customizable error responses come in three flavors:
# 1) plain text 2) local redirects 3) external redirects
#
# Some examples:
#ErrorDocument 500 "The server made a boo boo."
#ErrorDocument 404 /missing.html
#ErrorDocument 404 "/cgi-bin/missing_handler.pl"
#ErrorDocument 402 http://www.example.com/subscription_info.html
#

#
# MaxRanges: Maximum number of Ranges in a request before
# returning the entire resource, or one of the special
# values 'default', 'none' or 'unlimited'.
# Default setting is to accept 200 Ranges.
#MaxRanges unlimited

#
# EnableMMAP and EnableSendfile: On systems that support it,
# memory-mapping or the sendfile syscall may be used to deliver
# files.  This usually improves server performance, but must
# be turned off when serving from networked-mounted
# filesystems or if support for these functions is otherwise
# broken on your system.
# Defaults: EnableMMAP On, EnableSendfile Off
#
#EnableMMAP off
#EnableSendfile on

# Supplemental configuration
#
# The configuration files in the conf/extra/ directory can be
# included to add extra features or to modify the default configuration of
# the server, or you may simply copy their contents here and change as
# necessary.

# Server-pool management (MPM specific)
#Include conf/extra/httpd-mpm.conf

# Multi-language error messages
#Include conf/extra/httpd-multilang-errordoc.conf

# Fancy directory listings
#Include conf/extra/httpd-autoindex.conf

# Language settings
#Include conf/extra/httpd-languages.conf

# User home directories
#Include conf/extra/httpd-userdir.conf

# Real-time info on requests and configuration
#Include conf/extra/httpd-info.conf

# Virtual hosts
# SIDVINY
#Include conf/extra/httpd-vhosts.conf

# Local access to the Apache HTTP Server Manual
#Include conf/extra/httpd-manual.conf

# Distributed authoring and versioning (WebDAV)
#Include conf/extra/httpd-dav.conf

# Various default settings
#Include conf/extra/httpd-default.conf

# Configure mod_proxy_html to understand HTML4/XHTML1
<IfModule proxy_html_module>
Include conf/extra/proxy-html.conf
</IfModule>

# Secure (SSL/TLS) connections
#Include conf/extra/httpd-ssl.conf
#
# Note: The following must must be present to support
#       starting without SSL on platforms with no /dev/random equivalent
#       but a statically compiled-in mod_ssl.
#
<IfModule ssl_module>
SSLRandomSeed startup builtin
SSLRandomSeed connect builtin
</IfModule>


LoadFile "${CONDAROOT}/python310.dll"
LoadModule wsgi_module "${CONDAROOT}/lib/site-packages/mod_wsgi/server/mod_wsgi.cp310-win_amd64.pyd"

AddHandler wsgi-script .wsgi

WSGIPythonHome "${CONDAROOT}"
WSGIVerboseDebugging On
WSGIScriptReloading On
WSGIApplicationGroup %{GLOBAL}

Include conf/arcelormittal/httpd-vhosts.conf
````


## Appendix B: Final httpd-vhosts.conf after testing

````
<VirtualHost *:${SRVPORT}>
    #ServerName ws40968.sidmar.be
    #ServerAlias ws40968
    #ServerAlias localhost

    DocumentRoot "${PRYSENTROOT}"

    WSGIScriptAlias / "${PRYSENTROOT}/prysent/wsgi.py"


    ErrorLog "${SRVROOT}/logs/prysent.log"
    CustomLog "${SRVROOT}/logs/prysent.access.log" combined

    Alias /static "C:/Users/sidviny/Development/prysent/src/_html_cache"

    <Directory "${PRYSENTROOT}/prysent">
        <Files wsgi.py>
            Require all granted
        </Files>
    </Directory>

    <Directory "${PRYSENTROOT}">
        Require all granted
    </Directory>

    <Directory "C:/Users/sidviny/Development/prysent/src/_html_cache">
        Require all granted
    </Directory>
</VirtualHost>

<VirtualHost *:${SRVPORT}>
    #ServerName ws40968.sidmar.be
    #ServerAlias ws40968
    #ServerAlias localhost

    DocumentRoot "C:/Users/sidviny/Development/Support/flask_wsgi"

    WSGIScriptAlias / "C:/Users/sidviny/Development/Support/flask_wsgi/wsgi.py"


    ErrorLog "${SRVROOT}/logs/flask.log"
    CustomLog "${SRVROOT}/logs/flask.access.log" combined


    <Directory "C:/Users/sidviny/Development/Support/flask_wsgi">
        <Files wsgi.py>
            Require all granted
        </Files>
    </Directory>
</VirtualHost>

<VirtualHost *:${SRVPORT}>
    ServerName ws40968.sidmar.be
    ServerAlias ws40968
    ServerAlias localhost

    DocumentRoot "C:/Users/sidviny/Development/Support/coo_dashboard"

    WSGIScriptAlias / "C:/Users/sidviny/Development/Support/coo_dashboard/wsgi.py"


    ErrorLog "${SRVROOT}/logs/coo_dashboard.log"
    CustomLog "${SRVROOT}/logs/coo_dashboard.access.log" combined


    <Directory "C:/Users/sidviny/Development/Support/coo_dashboard">
        <Files wsgi.py>
            Require all granted
        </Files>
    </Directory>
</VirtualHost>
````


*(1) Actually, I first decided to download from ApacheHaus, but this version does not have the headers needed to build wsgi.  Which I did not yet knew at the first download.
